In [2]:
import os
import gc
import cv2
import time
import json
import paddle
import numpy as np
import pandas as pd
import paddle.nn as nn
import matplotlib.pyplot as plt
import paddle.nn.functional as F
from paddle.io import DataLoader, TensorDataset

In [2]:
class Identity(nn.Layer):
    """ Identity layer
    The output of this layer is the input without any change.
    This layer is used to avoid using 'if' condition in methods such as forward
    """
    def forward(self, x):
        return x


class PatchEmbedding(nn.Layer):
    """Patch Embedding
    Apply patch embedding (which is implemented using Conv2D) on input data.

    Attributes:
        image_size: image size
        patch_size: patch size
        num_patches: num of patches
        patch_embddings: patch embed operation (Conv2D)
    """
    def __init__(self,
                 image_size=224,
                 patch_size=16,
                 in_channels=3,
                 embed_dim=768):
        super().__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) * (image_size // patch_size)
        self.patch_embedding = nn.Conv2D(in_channels=in_channels,
                                         out_channels=embed_dim,
                                         kernel_size=patch_size,
                                         stride=patch_size)
    def forward(self, x):
        x = self.patch_embedding(x)
        x = x.flatten(2)  # [B, C, H, W] -> [B, C, h*w]
        x = x.transpose([0, 2, 1])  # [B, C, h*w] -> [B, h*w, C] = [B, N, C]
        return x


class Attention(nn.Layer):
    """ Attention module
    Attention module for ViT, here q, k, v are assumed the same.
    The qkv mappings are stored as one single param.

    Attributes:
        num_heads: number of heads
        attn_head_size: feature dim of single head
        all_head_size: feature dim of all heads
        qkv: a nn.Linear for q, k, v mapping
        scales: 1 / sqrt(single_head_feature_dim)
        out: projection of multi-head attention
        attn_dropout: dropout for attention
        proj_dropout: final dropout before output
        softmax: softmax op for attention
    """
    def __init__(self,
                 embed_dim,
                 num_heads,
                 attn_head_size=None,
                 qkv_bias=True,
                 dropout=0.,
                 attention_dropout=0.):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if attn_head_size is not None:
            self.attn_head_size = attn_head_size
        else:
            assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
            self.attn_head_size = embed_dim // num_heads
        self.all_head_size = self.attn_head_size * num_heads

        w_attr_1, b_attr_1 = self._init_weights()
        self.qkv = nn.Linear(embed_dim,
                             self.all_head_size * 3,  # weights for q, k, and v
                             weight_attr=w_attr_1,
                             bias_attr=b_attr_1 if qkv_bias else False)

        self.scales = self.attn_head_size ** -0.5

        w_attr_2, b_attr_2 = self._init_weights()
        self.out = nn.Linear(self.all_head_size,
                             embed_dim,
                             weight_attr=w_attr_2,
                             bias_attr=b_attr_2)

        self.attn_dropout = nn.Dropout(attention_dropout)
        self.proj_dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(axis=-1)
        self.attn_weights = None

    def _init_weights(self):
        weight_attr = paddle.ParamAttr(initializer=nn.initializer.TruncatedNormal(std=.02))
        bias_attr = paddle.ParamAttr(initializer=nn.initializer.Constant(0.0))
        return weight_attr, bias_attr

    def transpose_multihead(self, x):
        """[B, N, C] -> [B, N, n_heads, head_dim] -> [B, n_heads, N, head_dim]"""
        new_shape = x.shape[:-1] + [self.num_heads, self.attn_head_size]
        x = x.reshape(new_shape)  # [B, N, C] -> [B, N, n_heads, head_dim]
        x = x.transpose([0, 2, 1, 3])  # [B, N, n_heads, head_dim] -> [B, n_heads, N, head_dim]
        return x

    def forward(self, x):
        qkv = self.qkv(x).chunk(3, axis=-1)
        q, k, v = map(self.transpose_multihead, qkv)

        q = q * self.scales
        attn = paddle.matmul(q, k, transpose_y=True)  # [B, n_heads, N, N]
        attn = self.softmax(attn)
        attn = self.attn_dropout(attn)
        self.attn_weights = attn.detach()

        z = paddle.matmul(attn, v)  # [B, n_heads, N, head_dim]
        z = z.transpose([0, 2, 1, 3])  # [B, N, n_heads, head_dim]
        new_shape = z.shape[:-2] + [self.all_head_size]
        z = z.reshape(new_shape)  # [B, N, all_head_size]

        z = self.out(z)
        z = self.proj_dropout(z)
        return z


class Mlp(nn.Layer):
    """ MLP module
    Impl using nn.Linear and activation is GELU, dropout is applied.
    Ops: fc -> act -> dropout -> fc -> dropout

    Attributes:
        fc1: nn.Linear
        fc2: nn.Linear
        act: GELU
        dropout: dropout after fc
    """

    def __init__(self,
                 embed_dim,
                 mlp_ratio,
                 dropout=0.):
        super().__init__()
        w_attr_1, b_attr_1 = self._init_weights()
        self.fc1 = nn.Linear(embed_dim,
                             int(embed_dim * mlp_ratio),
                             weight_attr=w_attr_1,
                             bias_attr=b_attr_1)

        w_attr_2, b_attr_2 = self._init_weights()
        self.fc2 = nn.Linear(int(embed_dim * mlp_ratio),
                             embed_dim,
                             weight_attr=w_attr_2,
                             bias_attr=b_attr_2)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dropout)

    def _init_weights(self):
        weight_attr = paddle.ParamAttr(
            initializer=paddle.nn.initializer.TruncatedNormal(std=0.2))
        bias_attr = paddle.ParamAttr(
            initializer=paddle.nn.initializer.Constant(0.0))
        return weight_attr, bias_attr

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class TransformerLayer(nn.Layer):
    """Transformer Layer
    Transformer layer contains attention, norm, mlp and residual

    Attributes:
        embed_dim: transformer feature dim
        attn_norm: nn.LayerNorm before attention
        mlp_norm: nn.LayerNorm before mlp
        mlp: mlp modual
        attn: attention modual
    """
    def __init__(self,
                 embed_dim,
                 num_heads,
                 attn_head_size=None,
                 qkv_bias=True,
                 mlp_ratio=4.,
                 dropout=0.,
                 attention_dropout=0.,
                 droppath=0.):
        super().__init__()
        w_attr_1, b_attr_1 = self._init_weights()
        self.attn_norm = nn.LayerNorm(embed_dim,
                                      weight_attr=w_attr_1,
                                      bias_attr=b_attr_1,
                                      epsilon=1e-6)

        self.attn = Attention(embed_dim,
                              num_heads,
                              attn_head_size,
                              qkv_bias,
                              dropout,
                              attention_dropout)

        #self.drop_path = DropPath(droppath) if droppath > 0. else Identity()

        w_attr_2, b_attr_2 = self._init_weights()
        self.mlp_norm = nn.LayerNorm(embed_dim,
                                     weight_attr=w_attr_2,
                                     bias_attr=b_attr_2,
                                     epsilon=1e-6)

        self.mlp = Mlp(embed_dim, mlp_ratio, dropout)

    def _init_weights(self):
        weight_attr = paddle.ParamAttr(initializer=nn.initializer.Constant(1.0))
        bias_attr = paddle.ParamAttr(initializer=nn.initializer.Constant(0.0))
        return weight_attr, bias_attr

    def forward(self, x):
        h = x
        x = self.attn_norm(x)
        x = self.attn(x)
        #x = self.drop_path(x)
        x = x + h

        h = x
        x = self.mlp_norm(x)
        x = self.mlp(x)
        #x = self.drop_path(x)
        x = x + h

        return x


class Encoder(nn.Layer):
    """Transformer encoder
    Encoder encoder contains a list of TransformerLayer, and a LayerNorm.

    Attributes:
        layers: nn.LayerList contains multiple EncoderLayers
        encoder_norm: nn.LayerNorm which is applied after last encoder layer
    """
    def __init__(self,
                 embed_dim,
                 num_heads,
                 depth,
                 attn_head_size=None,
                 qkv_bias=True,
                 mlp_ratio=4.0,
                 dropout=0.,
                 attention_dropout=0.,
                 droppath=0.):
        super().__init__()
        # stochatic depth decay
        depth_decay = [x.item() for x in paddle.linspace(0, droppath, depth)]

        layer_list = []
        for i in range(depth):
            layer_list.append(TransformerLayer(embed_dim,
                                               num_heads,
                                               attn_head_size,
                                               qkv_bias,
                                               mlp_ratio,
                                               dropout,
                                               attention_dropout,
                                               depth_decay[i]))
        self.layers = nn.LayerList(layer_list)

        w_attr_1, b_attr_1 = self._init_weights()
        self.encoder_norm = nn.LayerNorm(embed_dim,
                                         weight_attr=w_attr_1,
                                         bias_attr=b_attr_1,
                                         epsilon=1e-6)

    def _init_weights(self):
        weight_attr = paddle.ParamAttr(initializer=nn.initializer.Constant(1.0))
        bias_attr = paddle.ParamAttr(initializer=nn.initializer.Constant(0.0))
        return weight_attr, bias_attr

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.encoder_norm(x)
        return x


class VisionTransformer(nn.Layer):
    """ViT transformer
    ViT Transformer, classifier is a single Linear layer for finetune,
    For training from scratch, two layer mlp should be used.
    Classification is done using cls_token.

    Args:
        image_size: int, input image size, default: 224
        patch_size: int, patch size, default: 16
        in_channels: int, input image channels, default: 3
        num_classes: int, number of classes for classification, default: 1000
        embed_dim: int, embedding dimension (patch embed out dim), default: 768
        depth: int, number ot transformer blocks, default: 12
        num_heads: int, number of attention heads, default: 12
        attn_head_size: int, dim of head, if none, set to embed_dim // num_heads, default: None
        mlp_ratio: float, ratio of mlp hidden dim to embed dim(mlp in dim), default: 4.0
        qkv_bias: bool, If True, enable qkv(nn.Linear) layer with bias, default: True
        dropout: float, dropout rate for linear layers, default: 0.
        attention_dropout: float, dropout rate for attention layers default: 0.
        droppath: float, droppath rate for droppath layers, default: 0.
        representation_size: int, set representation layer (pre-logits) if set, default: None
    """
    def __init__(self,
                 image_size=224,
                 patch_size=16,
                 in_channels=3,
                 num_classes=1000,
                 embed_dim=768,
                 depth=12,
                 num_heads=12,
                 attn_head_size=None,
                 mlp_ratio=4,
                 qkv_bias=True,
                 dropout=0.,
                 attention_dropout=0.,
                 droppath=0.,
                 representation_size=None):
        super().__init__()
        # create patch embedding
        self.patch_embedding = PatchEmbedding(image_size,
                                              patch_size,
                                              in_channels,
                                              embed_dim)
        # create posision embedding
        self.position_embedding = paddle.create_parameter(
            shape=[1, 1 + self.patch_embedding.num_patches, embed_dim],
            dtype='float32',
            default_initializer=paddle.nn.initializer.TruncatedNormal(std=.02))
        # create cls token
        self.cls_token = paddle.create_parameter(
            shape=[1, 1, embed_dim],
            dtype='float32',
            default_initializer=paddle.nn.initializer.TruncatedNormal(std=.02))
        self.pos_dropout = nn.Dropout(dropout)
        # create multi head self-attention layers
        self.encoder = Encoder(embed_dim,
                               num_heads,
                               depth,
                               attn_head_size,
                               qkv_bias,
                               mlp_ratio,
                               dropout,
                               attention_dropout,
                               droppath)
        # pre-logits
        if representation_size is not None:
            self.num_features = representation_size
            w_attr_1, b_attr_1 = self._init_weights()
            self.pre_logits = nn.Sequential(
                nn.Linear(embed_dim,
                          representation_size,
                          weight_attr=w_attr_1,
                          bias_attr=b_attr_1),
                nn.ReLU())
        else:
            self.pre_logits = Identity()

        # classifier head
        w_attr_2, b_attr_2 = self._init_weights()
        self.classifier = nn.Linear(embed_dim,
                                    num_classes,
                                    weight_attr=w_attr_2,
                                    bias_attr=b_attr_2)

    def _init_weights(self):
        weight_attr = paddle.ParamAttr(
            initializer=paddle.nn.initializer.Constant(1.0))
        bias_attr = paddle.ParamAttr(
            initializer=paddle.nn.initializer.Constant(0.0))
        return weight_attr, bias_attr

    def forward_features(self, x):
        x = self.patch_embedding(x)
        cls_tokens = self.cls_token.expand((x.shape[0], -1, -1))
        x = paddle.concat((cls_tokens, x), axis=1)
        x = x + self.position_embedding
        x = self.pos_dropout(x)
        x = self.encoder(x)
        x = self.pre_logits(x[:, 0]) # cls_token only
        return x

    def forward(self, x):
        x = self.forward_features(x)
        logits = self.classifier(x)
        return logits

In [10]:
class MetricsCalculator(object):  
    def __init__(self):  
        self.TP = 0  
        self.FP = 0  
        self.FN = 0  
        self.TN = 0
        self.y_trues = []
        self.y_pred_onehot = []
        self.y_preds_proba = []
  
    def update(self, y_true, y_pred, y_pred_onehot, y_pred_proba):  
        y_true = np.array(y_true, dtype='float64').reshape(-1, 1)  
        y_pred = np.array(y_pred, dtype='float64').reshape(-1, 1)
        y_pred_onehot = np.array(y_pred_onehot, dtype='float64').reshape(-1, 2)
        y_pred_proba = np.array(y_pred_proba, dtype='float64').reshape(-1, 2)

        self.y_trues.extend(y_true)
        self.y_pred_onehot.extend(y_pred_onehot)
        self.y_preds_proba.extend(y_pred_proba)  # Update the correct variable here
  
        # 假设 y_true 中 1 表示正类，0 表示负类  
        self.TP += np.sum((y_true == 1) & (y_pred == 1)) 
        self.FN += np.sum((y_true == 1) & (y_pred == 0))
        self.FP += np.sum((y_true == 0) & (y_pred == 1))
        self.TN += np.sum((y_true == 0) & (y_pred == 0))

    def calculate_brier_score(self):
        y_true = np.array(self.y_pred_onehot)
        y_pred_proba = np.array(self.y_preds_proba)  # Use the correct variable here

        # 计算 BS
        BS = np.mean((y_true - y_pred_proba) ** 2, axis=0)
        BS = BS[1]

        # 计算 BSS
        y_mean = np.mean(y_true, axis=0)
        reference_bs = np.mean((y_true - y_mean) ** 2, axis=0)
        reference_bs = np.mean((y_true[:, 1] - y_mean[1]) ** 2, axis=0)
        BSS = 1 - BS / reference_bs if reference_bs != 0 else 0

        return BS, BSS

    def calculate_metrics(self):  
        total = self.TP + self.FN + self.FP + self.TN  
  
        Accuracy = (self.TP + self.TN) / total if total > 0 else 0  
        Precision = self.TP / (self.TP + self.FP) if (self.TP + self.FP) > 0 else 0  
        Recall = self.TP / (self.TP + self.FN) if (self.TP + self.FN) > 0 else 0  
        FAR = self.FP / (self.FP + self.TP) if (self.FP + self.TP) > 0 else 0  
        TSS = Recall - (self.FP / (self.FP + self.TN)) if (self.FP + self.TN) > 0 else 0
        HSS = (2 * (self.TP * self.TN - self.FP * self.FN)) / ((self.TP + self.FN) * (self.FN + self.TN) + (self.TP + self.FP) * (self.FP + self.TN) + 1e-5)  
        
        BS, BSS = self.calculate_brier_score()
  
        metrics = {  
            'TP': self.TP,  
            'FP': self.FP,  
            'FN': self.FN,  
            'TN': self.TN,  
            'Accuracy': Accuracy,  
            'Precision': Precision,  
            'Recall': Recall,  
            'FAR': FAR,  
            'TSS': TSS,  
            'HSS': HSS,
            'Brier Score (BS)': BS,
            'Brier Skill Score (BSS)': BSS
        }  
        return metrics 

In [4]:
def data_preprocess(data_path, chunk_size=1000):
    labels_list = []
    images_list = []

    # Process data in chunks to save memory
    for chunk in pd.read_csv(data_path, header=None, usecols=[3] + list(range(4, 16388)), chunksize=chunk_size):
        labels_chunk = chunk.iloc[:, 0].values
        images_chunk = chunk.iloc[:, 1:].values.astype('float32').reshape(-1, 128, 128)
        
        # Convert labels to binary
        label_mapping = {'N': 0, 'C': 0, 'M': 1, 'X': 1}
        labels_chunk = np.vectorize(label_mapping.get)(labels_chunk).astype('float32').reshape(-1, 1)
        
        # Resize and preprocess images
        images_resized = np.array([cv2.resize(img, (224, 224), interpolation=cv2.INTER_LINEAR) for img in images_chunk])
        images_resized = np.stack([images_resized]*3, axis=-1)  # Convert to 3 channels
        images_resized = images_resized.transpose([0, 3, 1, 2])  # Change to (num_samples, channels, height, width)
        images_resized /= 4000.0  # Normalize

        labels_list.append(labels_chunk)
        images_list.append(images_resized)

        # Clear memory
        del chunk, labels_chunk, images_chunk, images_resized
        gc.collect()

    labels = np.vstack(labels_list)
    images = np.vstack(images_list)

    # Clear memory
    del labels_list, images_list
    gc.collect()

    return images, labels

In [5]:
paddle.seed(40)
np.random.seed(42)
paddle.device.cuda.empty_cache()
weight = paddle.to_tensor([0.569, 4.120], dtype='float32')

W0819 21:22:40.576051 31808 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
W0819 21:22:40.577553 31808 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.


In [ ]:
batch_val = 20
Train_Loss = []
Val_Loss = []

for dataset_id in range(10):
    best_TSS = -1
    # Data preprocessing
    train_data, train_label = data_preprocess(f'dataset/DATA/feature/group9_Data2_image/{dataset_id}Train.csv')
    val_data, val_label = data_preprocess(f'dataset/DATA/feature/group9_Data2_image/Sift/{dataset_id}Val.csv')

    paddle.device.cuda.empty_cache()
    gc.collect()

    train_dataset = TensorDataset([train_data, train_label])
    val_dataset = TensorDataset([val_data, val_label])
    train_loader = DataLoader(train_dataset, batch_size=712, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)

    model = VisionTransformer(image_size=224,
                              patch_size=16,
                              in_channels=3,
                              num_classes=1000,
                              embed_dim=192,
                              depth=12,
                              num_heads=3,
                              attn_head_size=None,
                              mlp_ratio=4.0,
                              qkv_bias=True,
                              dropout=0.01,
                              attention_dropout=0.01,
                              droppath=0.,
                              representation_size=None)
    param = paddle.load('vit_tiny_patch16_224.pdparams')
    model.load_dict(param)
    model.classifier = paddle.nn.Linear(192, 2)  

    scheduler = paddle.optimizer.lr.LinearWarmup(
        learning_rate=1e-5, 
        warmup_steps=20, 
        start_lr=1e-7, 
        end_lr=1e-5, 
        verbose=False)
    opt = paddle.optimizer.AdamW(
        learning_rate=scheduler, 
        weight_decay=0.15, 
        parameters=model.parameters())

    for i in range(30):
        batch_id = 0
        total_loss = 0
        model.train()

        for img, label in train_loader:
            batch_id += 1
            label = label.astype('int32')

            pred = model(img)
            loss = F.cross_entropy(pred, label, weight=weight)
            total_loss += loss.item()

            opt.clear_gradients()
            loss.backward()
            opt.step()

            if batch_id % batch_val == 0:
                print(f'Epoch: {i+1}')
                print(f'Train_Loss: {total_loss / batch_val}')
                Train_Loss.append(total_loss / batch_val)
                total_loss = 0

                TSS = []
                val_batch_id = 0
                calculator = MetricsCalculator()
                model.eval()

                with paddle.no_grad():
                    for img, label in val_loader:
                        val_batch_id += 1
                        pred = model(img)
                        label = label.astype('int32')
                        loss = F.cross_entropy(pred, label, weight=weight)
                        total_loss += loss.item()
                        pred_label = paddle.argmax(pred, axis=-1)
                        calculator.update(y_true=label, y_pred=pred_label)

                val_loss_avg = total_loss / val_batch_id
                print(f'Val_Loss: {val_loss_avg}')
                Val_Loss.append(val_loss_avg)
                total_loss = 0

                metric = calculator.calculate_metrics()
                if best_TSS < metric['TSS']:
                    best_TSS = metric['TSS']
                    paddle.save(model.state_dict(), f'model/Transformer_sift/ViT_{dataset_id}.pdparam')
                print(metric)
                print('Mean TSS: {}   Best_TSS:{}\n'.format(metric['TSS'], best_TSS))

                # Clear CUDA cache and collect garbage
                paddle.device.cuda.empty_cache()
                gc.collect()

        scheduler.step()

    loss_folder = 'Loss/ViT_sift/'
    if not os.path.exists(loss_folder):
        os.makedirs(loss_folder)
    np.save(loss_folder + f'Train_Loss_{dataset_id}.npy', np.array(Train_Loss))
    np.save(loss_folder + f'Val_Loss_{dataset_id}.npy', np.array(Val_Loss))    

    # Clear CUDA cache and collect garbage after each dataset
    paddle.device.cuda.empty_cache()
    gc.collect()

## Test

In [11]:
for dataset_id in range(10):
    test_data, test_label = data_preprocess(f'dataset/DATA/feature/group9_Data2_image/{dataset_id}Test.csv')
    test_data = test_data[39::40, ...]
    test_label = test_label[39::40, ...]
    test_dataset = TensorDataset([test_data, test_label])
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
    model = VisionTransformer(image_size=224,
                              patch_size=16,
                              in_channels=3,
                              num_classes=2,
                              embed_dim=192,
                              depth=12,
                              num_heads=3,
                              attn_head_size=None,
                              mlp_ratio=4.0,
                              qkv_bias=True,
                              dropout=0.01,
                              attention_dropout=0.01,
                              droppath=0.,
                              representation_size=None)
    calculator = MetricsCalculator()
    param = paddle.load(f'model/Transformer/ViT_{dataset_id}.pdparam')
    model.load_dict(param)
    model.eval()

    TSS = []
    with paddle.no_grad():
        for img, label in test_loader:
            pred = model(img)
            label = label.astype('long')
            pred_label = paddle.argmax(pred, axis=-1)
            pred_onehot = F.one_hot(label, 2)
            pred_proba = F.softmax(pred, axis=-1)
            calculator.update(label, pred_label.detach().cpu().numpy(), pred_onehot.detach().cpu().numpy(), pred_proba.detach().cpu().numpy())
    metric = calculator.calculate_metrics()
    print(metric)

    metric_folder = 'Metrics_timestep_40/ViT_BSS/'
    if not os.path.exists(metric_folder):
        os.makedirs(metric_folder)
    data_serializable = {k: int(v) if isinstance(v, np.integer) else v for k, v in metric.items()} 
    with open(metric_folder + f'dataset_{dataset_id}.json', 'w', encoding='utf-8') as f:  
        json.dump(data_serializable, f, ensure_ascii=True, indent=4)    

{'TP': 27, 'FP': 21, 'FN': 8, 'TN': 119, 'Accuracy': 0.8342857142857143, 'Precision': 0.5625, 'Recall': 0.7714285714285715, 'FAR': 0.4375, 'TSS': 0.6214285714285714, 'HSS': 0.5454545449660058, 'Brier Score (BS)': 0.13057568534762004, 'Brier Skill Score (BSS)': 0.1839019665773749}
{'TP': 30, 'FP': 22, 'FN': 5, 'TN': 118, 'Accuracy': 0.8457142857142858, 'Precision': 0.5769230769230769, 'Recall': 0.8571428571428571, 'FAR': 0.4230769230769231, 'TSS': 0.7, 'HSS': 0.5921450145946094, 'Brier Score (BS)': 0.1132763539787552, 'Brier Skill Score (BSS)': 0.2920227876327801}
{'TP': 29, 'FP': 24, 'FN': 6, 'TN': 116, 'Accuracy': 0.8285714285714286, 'Precision': 0.5471698113207547, 'Recall': 0.8285714285714286, 'FAR': 0.4528301886792453, 'TSS': 0.6571428571428573, 'HSS': 0.5508982031215585, 'Brier Score (BS)': 0.14317451286201258, 'Brier Skill Score (BSS)': 0.10515929461242157}
{'TP': 29, 'FP': 33, 'FN': 6, 'TN': 107, 'Accuracy': 0.7771428571428571, 'Precision': 0.46774193548387094, 'Recall': 0.82857

In [4]:
read_folder = 'Metrics_timestep_40/ViT_BSS/'
read_file = sorted(os.listdir(read_folder))[:]
TSS = []
Accuracy = []
Recall = []
FAR = []
Percision = []
HSS = []
BSS = []
BS = []
FPR = []
for r_file in read_file:
    r_file = os.path.join(read_folder, r_file)
    with open(r_file, 'r') as f:
        data = json.load(f)
        print(data)
        FP = data['FP']
        TN = data['TN']
        FPR.append(FP / (FP + TN))
        TSS.append(data['TSS'])
        Accuracy.append(data['Accuracy'])
        Percision.append(data['Precision'])
        Recall.append(data['Recall'])
        FAR.append(data['FAR'])
        HSS.append(data['HSS'])
        BSS.append(data['Brier Skill Score (BSS)'])
        BS.append(data['Brier Score (BS)'])
print(len(Recall))
print('Accuracy:', np.mean(Accuracy), np.std(Accuracy))
print('Percision:', np.mean(Percision), np.std(Percision))
print('Recall:', np.mean(Recall), np.std(Recall))
print('FAR:', np.mean(FAR), np.std(FAR))
print('FPR:', np.mean(FPR), np.std(FPR))
print('TSS:', np.mean(TSS), np.std(TSS))
print('HSS:', np.mean(HSS), np.std(HSS))
print('BSS:', np.mean(BSS), np.std(BSS))
print('BS :', np.mean(BS), np.std(BS))

{'TP': 27, 'FP': 21, 'FN': 8, 'TN': 119, 'Accuracy': 0.8342857142857143, 'Precision': 0.5625, 'Recall': 0.7714285714285715, 'FAR': 0.4375, 'TSS': 0.6214285714285714, 'HSS': 0.5454545449660058, 'Brier Score (BS)': 0.13057568534762004, 'Brier Skill Score (BSS)': 0.1839019665773749}
{'TP': 30, 'FP': 22, 'FN': 5, 'TN': 118, 'Accuracy': 0.8457142857142858, 'Precision': 0.5769230769230769, 'Recall': 0.8571428571428571, 'FAR': 0.4230769230769231, 'TSS': 0.7, 'HSS': 0.5921450145946094, 'Brier Score (BS)': 0.1132763539787552, 'Brier Skill Score (BSS)': 0.2920227876327801}
{'TP': 29, 'FP': 24, 'FN': 6, 'TN': 116, 'Accuracy': 0.8285714285714286, 'Precision': 0.5471698113207547, 'Recall': 0.8285714285714286, 'FAR': 0.4528301886792453, 'TSS': 0.6571428571428573, 'HSS': 0.5508982031215585, 'Brier Score (BS)': 0.14317451286201258, 'Brier Skill Score (BSS)': 0.10515929461242157}
{'TP': 29, 'FP': 33, 'FN': 6, 'TN': 107, 'Accuracy': 0.7771428571428571, 'Precision': 0.46774193548387094, 'Recall': 0.82857